#### Consultancy Project
A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

Name : Name of the latest contact at Company
Age: Customer Age
Total_Purchase: Total Ads Purchased
Account_Manager: Binary 0=No manager, 1= Account manager assigned
Years: Totaly Years as a customer
Num_sites: Number of websites that use the service.
Onboard_date: Date that the name of the latest contact was onboarded
Location: Client HQ Address
Company: Name of Client Company
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import pandas as pd

In [2]:
#import the necessary libraries
from pyspark.sql import SparkSession

In [3]:
#import the isnan function from the column package
from pyspark.sql.functions import (count, when, isnull, month, year)

In [4]:
#starting the spark session
spark = SparkSession.builder.appName('customerchurn').getOrCreate()

In [5]:
#load the dataset
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

AnalysisException: 'Path does not exist: file:/C:/Users/galipbak/iCloudDrive/CODING/GITHUB/data_analytics_playground/customer_churn.csv;'

#### Exploratory analysis

In [ ]:
#analyzing each column
data.printSchema()

In [ ]:
#print a couple of rows for further analysis
for row in data.head(5):
    print(row,'\n')

In [ ]:
def null_counter(dataframe): 
    '''Accepts a spark dataframe, and returns the number of null values in each column:
    ex:
    input:  null_counter(dataframe_with_missingvalues)
     output: 
            The number of missing values in  Id  column :  0
            The number of missing values in  Name  column :  2 
    '''
    for column in dataframe.columns:
        print('The number of missing values in ',column, ' column : ',
              dataframe.select(count(when(isnull(column),column)).alias(column)).collect()[0][0]
              )

In [ ]:
null_counter(data)

#### Data seems complete
#### There are no missing values in the data.
#### Now let's check the categorical values

In [ ]:
data.printSchema()

##### Names column will not affect the analysis
##### Company Location and Company address reveal the same data. We will drop the address
##### Onboard_date needs to be modifed only to year and month
##### Company needs to be one-hot-encoded

In [ ]:
#create a list of all the stages of workflow for the pipeline
stages = []

###  Convert the onboard date to month duration

In [ ]:
#import the date time module
import datetime

In [ ]:
#converting the current date (3/15/2020) to months
currentmonth = datetime.datetime.now().year*12 + datetime.datetime.now().month

In [ ]:
#import the lit function from pyspark to be able to add a constant as a column
from pyspark.sql.functions import lit

#adding the current month to the the data frame
data = data.withColumn('Current_Month', lit(currentmonth))

In [ ]:
#the duration which the latest contact stayed in contact could be important. So:

#calculating the date of the onboard_date in terms of month
data = data.withColumn('Onboard_Month', ( year('Onboard_date')*12  +  month('Onboard_date') ).alias('Onboard_month'))

#finding the difference between the current month and the onboard_date
data = data.withColumn('Onboard_Date_Difference', data['Current_Month'] - data['Onboard_Month'])

In [ ]:
data.columns

In [ ]:
#finalize the dataset
data_in_progress = data.select(
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Company',
 'Onboard_Date_Difference', 
 'Churn')

### Handle the categorical variables

In [ ]:
#importing the indexer and the onehotencoder
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
#first create the company indexer
company_indexer = StringIndexer(inputCol='Company', outputCol='Company_indexed')
stages += [company_indexer] #adding to the stages for the pipeline

#secondly, create a company encoder
company_encoder = OneHotEncoder(inputCol='Company_indexed', outputCol='Company_encoded')
stages += [company_encoder]  #adding to the stages for the pipeline

### Build the assembler

In [ ]:
#importing the VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [ ]:
#building the assembler
assembler = VectorAssembler(inputCols=['Age','Total_Purchase',
                                        'Account_Manager','Years',
                                        'Num_Sites','Company_encoded',
                                        'Onboard_Date_Difference'],
                            outputCol='features')
stages += [assembler]

### Building the workflow pipeline

In [ ]:
#import the pipeline
from pyspark.ml import Pipeline

In [ ]:
#create the pipeline
pipeline = Pipeline(stages = stages)

In [ ]:
final_data = pipeline.fit(data_in_progress).transform(data_in_progress)

In [ ]:
final_data.printSchema()

### Create the train and test datasets

In [ ]:
#splitting the data
train_data, test_data = final_data.randomSplit([0.7, 0.3])

### Build the regression model

In [ ]:
#importing the logistic regression
from pyspark.ml.classification import LogisticRegression

In [ ]:
#start the log_regression object
log_reg = LogisticRegression(labelCol='Churn')

### Build the model

In [ ]:
#train the model
log_reg_model = log_reg.fit(final_data)

### Evaluate the model

In [ ]:
#introduce the test data to the model
results = log_reg_model.transform(test_data)

In [ ]:
#checking the output
results.columns

In [ ]:
#review the results dataframe using pandas pandas for better formatting
pd.DataFrame(data=results.take(3),columns=results.columns).T

### Use the evaluation metrics for further analysis

In [ ]:
#importing the evaluation metrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
#build an evaluator
evaluate = BinaryClassificationEvaluator(labelCol='Churn')

In [ ]:
#calculate the AUC
AUC = evaluate.evaluate(results)

In [ ]:
AUC

### Our model classified all data correctly

### Now, we will do further analysis before deploying our model